In [1]:
from urllib.request import urlopen
from bs4 import BeautifulSoup
import re

#trang web lấy dữ liệu
URL = 'https://vnexpress.net'


def getLink(url):
    links = []
    html = urlopen(url).read()
    soup = BeautifulSoup(html, 'html.parser')

    #lấy đường dẫn hiện tại
    urls = set(re.findall(
        r'(?:http|https|ftp):\/\/(?:[\w_-]+(?:(?:\.[\w_-]+)+))(?:[\w.,@?^=%&:/~+#-]*[\w@?^=%&/~+#-])', str(soup)))

    for url in urls:
        #chỉ lấy web chứa báo
        if url != URL and url.startswith(URL) and not url.startswith(URL + '/tac-gia') and url.endswith('.html'):
            links.append(url)
    
    return links

In [2]:
getLink(URL)

['https://vnexpress.net/ten-hag-man-utd-co-co-hoi-lon-vao-top-4-4552320.html',
 'https://vnexpress.net/giang-sinh-cua-mot-linh-muc-cong-giao-4552259.html',
 'https://vnexpress.net/tien-so-dung-truoc-nga-ba-duong-4550654.html',
 'https://vnexpress.net/muc-tieu-khong-tron-ven-cua-ong-zelensky-trong-chuyen-tham-my-4551637.html',
 'https://vnexpress.net/nguoi-ha-noi-tp-hcm-do-ra-duong-don-giang-sinh-4552288.html',
 'https://vnexpress.net/an-do-kho-soan-vi-the-cong-xuong-the-gioi-cua-trung-quoc-4547493.html',
 'https://vnexpress.net/thuong-tet-o-ha-noi-cao-nhat-400-trieu-dong-4552240.html',
 'https://vnexpress.net/myanmar-thua-nguoc-singapore-o-aff-cup-2022-4552281.html',
 'https://vnexpress.net/be-quyen-toi-buon-vi-phu-long-khan-gia-4552245.html',
 'https://vnexpress.net/hoa-hau-thanh-thuy-toi-xung-dang-dang-quang-4552140.html',
 'https://vnexpress.net/niem-vui-be-bong-4552034.html',
 'https://vnexpress.net/cuu-bi-thu-tinh-uy-dong-nai-bi-de-nghi-10-11-nam-tu-4552130.html',
 'https://vnexpr

In [3]:
categories = ['thoi-su','kinh-doanh','khoa-hoc','giai-tri','the-thao','phap-luat','giao-duc','suc-khoe','doi-song','du-lich','so-hoa']
articleURL = {}

for category in categories:
    articleURL[category] = []

    for i in range(1,11):
        url = '{}/{}-p{}'.format(URL, category, str(i))
        articleURL[category] += getLink(url)

    articleURL[category] = list(set(articleURL[category]))

    print(category, len(articleURL[category]))

gocNhin = ['binh-luan-nhieu','covid-19','chinh-tri-chinh-sach','y-te-suc-khoe','kinh-doanh-quan-tri','giao-duc-tri-thuc','moi-truong','van-hoa-loi-song']
articleURL['goc-nhin'] = []

for subCate in gocNhin:
    url = '{}/{}/{}'.format(URL, 'goc-nhin', subCate)
    articleURL['goc-nhin'] += getLink(url)

articleURL['goc-nhin'] = list(set(articleURL['goc-nhin']))
print('goc-nhin', len(articleURL['goc-nhin']))


thoi-su 302
kinh-doanh 320
khoa-hoc 308
giai-tri 306
the-thao 312
phap-luat 309
giao-duc 308
suc-khoe 320
doi-song 310
du-lich 322
so-hoa 312
goc-nhin 91


In [4]:
def getContent(url):
    content = ''
    html = urlopen(url).read()
    soup = BeautifulSoup(html, 'html.parser')

    divContent = soup.select('.page-detail .container')
    if len(divContent) > 0 :
        divContent = divContent[0]

        description = divContent.find_all('p', {'class': 'description'})
        if len(description) > 0:
            description = description[-1]
            textDescription = description.get_text()
            location = description.find('span', {'class': 'location-span'})
            if location is not None:
                content = textDescription[len(location.get_text()):]
            else:
                content = textDescription

        detail = divContent.find('article', {'class': 'fck_detail'})
        if detail is not None:
            p_normal = detail.find_all('p', {'class': 'Normal'})
            if len(p_normal) > 0:
                for p in p_normal:
                    p_text = p.get_text()
                    if p_text != '' and not p_text.startswith('>>') and p_text[-1] in ['.','!','?']:
                        content += ' ' + p.get_text()

    return re.sub(r'\n',' ', content)

In [5]:
getContent(articleURL['giao-duc'][0])

'Cố gắng học theo công thức, lệ thuộc bài viết mẫu, dùng từ vựng cao siêu, là những sai lầm thường gặp khi thí sinh học IELTS Writing. Writing (Viết) là một trong bốn kỹ năng của bài thi IELTS (hệ thống kiểm tra Anh ngữ quốc tế, gồm: Nghe, Nói, Đọc, Viết). Bài thi Writing gồm 2 phần: Task 1 - Bài viết miêu tả và tóm tắt thông tin trong biểu đồ, hình vẽ và Task 2 - Bài viết nghị luận (Argumentative) về một chủ đề cho trước. Đối với nhiều thí sinh, đây là phần thi khó nhất trong bốn kỹ năng. Từng thi nhiều lần đạt IELTS 8.5, anh Đinh Quang Tùng, ở TP HCM, chia sẻ 5 sai lầm thường gặp khi học IELTS Writing. Nhiều bạn lầm tưởng rằng bài thi IELTS Writing có những công thức giúp đạt được các band điểm khác nhau. - The widespread problem of + vấn đề trong bài + has long been a major topic of concern in society. - Along with the development of society, + vấn đề trong bài + has become a topic of heated debate/a topic of controversy/a controversial topic. Công thức trên không có vấn đề gì về ng

In [6]:
import os

sentence = []
uppercase = 'AÁÀẢÃẠÂẤẦẨẪẬĂẮẰẲẴẶBCDĐEÉÈẺẼẸÊẾỀỂỄỆFGHIÍÌỈĨỊJKLMNOÓÒỎÕỌÔỐỒỔỖỘƠỚỜỞỠỢPQRSTUÚÙỦŨỤƯỨỪỬỮỰVWXYÝỲỶỸỴZ'

for category in articleURL.keys():
    
    currentPath = './Data/Categories/' + category + '/'
    os.mkdir(currentPath)
    
    count = 0
    for i in range(len(articleURL[category])):
        content = getContent(articleURL[category][i])
        if content != '':
            count += 1
            f = open('{}/{}_{}.txt'.format(currentPath, category, str(count)), 'w', encoding='utf-8')

            lines = re.sub(r'(?<=[.?!])(?=[\'"`]*[' + uppercase + '-])', ' ', content)

            lines = re.split(r'(?<=[.?!])\s+(?=[\'"`]*[' + uppercase + '0-9-])', content)

            for line in lines:
                f.write(line + '\n')
                sentence.append(line)
            f.close()
    
    print(category, count)

sentence = list(set(sentence))
print('Tổng số câu tách được : ', len(sentence))


thoi-su 300
kinh-doanh 317
khoa-hoc 308
giai-tri 306
the-thao 305
phap-luat 309
giao-duc 306
suc-khoe 320
doi-song 287
du-lich 319
so-hoa 309
goc-nhin 91
Tổng số câu tách được :  78565


In [7]:
fileCorpus = open('./corpus.txt','w', encoding='utf-8')

count = 0 
for sent in sentence:
    sent = re.sub(r'\u200b', '', sent)
    fileCorpus.write(sent.strip() + '\n')
    count += 1
fileCorpus.close()

print(count)

78565
